# Kobe bryant shot selection

## 1. Data load

In [ ]:
import pandas as pd

data = pd.read_csv("data.csv")

In [ ]:
print("total number of shots =",len(data))
data.head()

In [ ]:
for col in data.columns:
    print(col)

In [ ]:
# 'shot_made_flag'가 shot이 들어갔는지의 여부 == target_varaible
data.shot_made_flag.head()

- 실제 진행되었던 competition에서는 NaN을 예측하여 제출하는 형식
- NaN에 대한 정답 label이 없기 때문에, NaN이 아닌 shots 데이터만 사용!


In [ ]:
# NaN 데이터 분리
data = data[data.shot_made_flag.isnull()==False] # 같은 표현 : data = data.dropna()
print(data.shot_made_flag.head())

# data 개수 확인
print("total number of labeled shots = ",len(data))

### 만약 정답 label이 있는 데이터를 train과 test를 나누려면

In [ ]:
from sklearn.model_selection import train_test_split

X = data.ix[:,data.columns != 'shot_made_flag'].copy() # 'shot_made_flag'를 제외한 모든 columns를 선택
y = data.shot_made_flag.copy()

# train_test_split
trn_x, tst_x, trn_y, tst_y = train_test_split(X, y, test_size=0.3, random_state =42)

In [ ]:
# 나눠진 data 개수 체크
print('# of trn_x = ',len(trn_x))
print('# of trn_y = ',len(trn_y))
print('# of tst_x = ',len(tst_x))
print('# of tst_y = ',len(tst_y))

## 2. Data Exploration and Visualization
- 데이터에는 어떤 feature들이 있는지 확인해보자

In [ ]:
for i, feature in enumerate(trn_x.columns):
    print(i,feature)

## 2-1 Action type

In [ ]:
action_shot = []
for action in set(trn_x.action_type):
    action_shot.append((action, len(trn_x[trn_x.action_type == action]), sum(trn_y[trn_x.action_type == action])))

In [ ]:
# action_shot list 구조 확인 : 'action_type', '해당 action type으로 쏜 모든 슛', '해당 action type으로 들어간 슛'
action_shot[1]

In [ ]:
# 슛 성공률에 따라 정렬 - lambda function
action_shot.sort(key= lambda x: x[2]/x[1], reverse=True)

for action, total_shots, shots in action_shot:
    print("Action type = {:35s}# of shots = {:>8.2f}\t accuracy = {:>3.2f}".format(action, total_shots, shots/total_shots)) 

- 살펴봐야 하는 action type이 너무 많음
- Jump shot이 제일 많고, Dunk 관련 action type이 성공률이 높음
- 'combined shot type을 살펴 보자

## 2-2 Combined shot type

In [ ]:
combined_shot = []
for combined in set(trn_x.combined_shot_type):
    combined_shot.append((combined, len(trn_x[trn_x.combined_shot_type == combined]), sum(trn_y[trn_x.combined_shot_type == combined])))
    
# 슛 성공률에 따라 정렬
combined_shot.sort(key= lambda x: x[2]/x[1], reverse=True)
   
for action, total_shots, shots in combined_shot:
    print("Combined action type = {:10s}# of shots = {:>8.2f}\t accuracy = {:>3.2f}".format(action, total_shots, shots/total_shots))

In [ ]:
# merge trn_x and trn_y
trn_total = trn_x.copy()
trn_total['shot_made_flag'] = trn_y.copy()

In [ ]:
?plt.plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# combined shot types
groups = trn_total.groupby('combined_shot_type')
court_scale = 14
alphas = []
colors = ['blue', 'gray', 'red', 'cyan', 'purple', 'yellow']

for u in [i[0] for i in groups]:
    d = len(trn_total.loc[trn_total.combined_shot_type == u, 'combined_shot_type'])
    alphas.append(np.log1p(d))

# 실제 농구 코트 비율 28 : 15
fig, ax = plt.subplots(2, 3, figsize=(court_scale, court_scale*(18/15)))

for i, ((name, group), alp, col) in enumerate(zip(groups, alphas, colors)):
    x = i//3
    y = i%3
    ax[x, y].set_autoscale_on(False)
    ax[x, y].margins(0.05) # Optional, just adds 5% padding to the autoscaling
    ax[x, y].axis([-300,300,0,750])
    ax[x, y].plot(group.loc_x, group.loc_y, marker='.', linestyle='', ms=12, label=name, alpha=alp, color=col)
    ax[x, y].legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# combined shot types
groups = trn_total.groupby('combined_shot_type')
court_scale = 14
alphas = []

for u in [i[0] for i in groups]:
    d = len(trn_total.loc[trn_total.combined_shot_type == u, 'combined_shot_type'])
    alphas.append(np.log1p(d))

# 실제 농구 코트 비율 28 : 15
fig, ax = plt.subplots(2, 3, figsize=(court_scale, court_scale*(18/15)))

for i, ((name, group), alp) in enumerate(zip(groups, alphas)):
    # 순서대로 plot 위치를 정해줌
    x = i//3
    y = i%3
    
    # 각각의 plot의 parameter setting
    ax[x, y].set_autoscale_on(False)
    ax[x, y].margins(0.05) # Optional, just adds 5% padding to the autoscaling
    ax[x, y].axis([-300,300,0,750])
    
    # 파랑이 성공, 빨강이 실패
    ax[x, y].plot(group[group.shot_made_flag == 1].loc_x, group[group.shot_made_flag == 1].loc_y, marker='.', 
                                                                                linestyle='', ms=12, label='Success', alpha=alp, color='blue')
    ax[x, y].plot(group[group.shot_made_flag != 1].loc_x, group[group.shot_made_flag != 1].loc_y, marker='.', 
                                                                                linestyle='', ms=12, label='Fail', alpha=alp, color='red')
    ax[x, y].legend()
    
    # title 지정
    score =  ' ' + str(len(group[group.shot_made_flag == 1])) + '/' +str(len(group[group.shot_made_flag != 1]))
    ax[x, y].title.set_text(name + score) # 성공한 슛/실패한 슛




## 2-3 game_event_id, game_id

In [ ]:
for i in range(5):
    print(X.iloc[i].game_event_id, X.iloc[i].game_id)

- game_id는 한 경기당 부여되는 id
- game_event_id는 한 경기에 발생하는 event(i.e. shot) id

## 2-4 lat-lon, loc_x, loc_y 

In [ ]:
# plot 전체 size를 설정
plt.figure(figsize=(28,15))
alpha = 0.05 # plot 하나의 진하기 결정, 클수록 진한 포인트

# loc_x and loc_y
plt.subplot(121) # 1X2 map에 1번째 그림
plt.scatter(trn_x.loc_x, trn_x.loc_y, color='b', alpha=alpha)
plt.title('loc_x and loc_y')

# lat and lon
plt.subplot(122) # 1X2 map에 2번째 그림
plt.scatter(trn_x.lon, trn_x.lat, color='g', alpha=alpha)
plt.title('lat and lon')


- lat-lon과 loc_x, loc_y는 shot의 위치를 나타내는 동일 정보

### 2-5 shot distance - 성공률

In [ ]:
# Shooting accuracy with shot distance
def get_acc(df, against):
    ct = pd.crosstab(df.shot_made_flag, df[against]).apply(lambda x:x/x.sum(), axis=0)
    x, y = ct.columns, ct.values[1, :]
    plt.figure(figsize=(20, 5))
    plt.plot(x, y)
    plt.xlabel(against)
    plt.ylabel('% shots made')
    plt.savefig(against + '_vs_accuracy.png')

In [ ]:
get_acc(trn_total, 'shot_distance')

## 2-6 remaining time - 성공률

In [ ]:
# What can we learn from time?
get_acc(trn_total, 'seconds_remaining')

In [ ]:
get_acc(trn_total, 'minutes_remaining')

### 초와 분만 가지고 보는건 의미가 없음
- remaing_seconds + remaing_minutes를 seconds_from_period_end으로 변환
- seconds_from_period_end = 60 * remaing_minutes + remaing_seconds

In [ ]:
trn_total['seconds_from_period_end'] = 60 * trn_total['minutes_remaining'] + trn_total['seconds_remaining']
get_acc(trn_total, 'seconds_from_period_end')

In [ ]:
get_acc(trn_total, 'period')

## 2-7 시즌에 따른 성공률

In [ ]:
print(trn_total.season.unique())

# 연도만 분리
trn_total['season_start_year'] = trn_total.season.str.split('-').str[0]

# integer로 변환
trn_total['season_start_year'] = trn_total['season_start_year'].astype(int)

get_acc(trn_total, 'season_start_year')

- 마지막 season에는 유난히 슛 성공률이 낮은 것을 확인할 수 있음

## 2-8 Shot zone

In [ ]:
plt.figure(figsize=(7.5,14))

for zone in set(trn_x.shot_zone_area):
    plt.plot(trn_x[trn_x.shot_zone_area == zone].loc_x, trn_x[trn_x.shot_zone_area == zone].loc_y, marker='.', linestyle='', alpha = 0.5, label=zone)
    plt.legend()

### zone에 따라 success/fail 분포 살펴보기

In [ ]:
def group_sf(data, category):
    # combined shot types
    groups = data.groupby(category)
    n = len(set(data[category]))
    k = (n+1)//2
    # 실제 농구 코트 비율 28 : 15
    fig, ax = plt.subplots(2, k, figsize=(court_scale, court_scale*(20/15)))

    for i, (name, group) in enumerate(groups):
        x = i//k
        y = i%k
        ax[x, y].set_autoscale_on(False)
        ax[x, y].margins(0.05) # Optional, just adds 5% padding to the autoscaling
        ax[x, y].axis([-300,300,0,750])

        # 파랑이 성공, 빨강이 실패
        ax[x, y].plot(group[group.shot_made_flag == 1].loc_x, group[group.shot_made_flag == 1].loc_y, marker='.', 
                                                                                    linestyle='', ms=12, label='Success', alpha=0.5, color='b')
        ax[x, y].plot(group[group.shot_made_flag != 1].loc_x, group[group.shot_made_flag != 1].loc_y, marker='.', 
                                                                                    linestyle='', ms=12, label='Fail', alpha=0.5, color='r')
        ax[x, y].legend()
        # title 지정
        score =  ' ' + str(len(group[group.shot_made_flag == 1])) + '/' +str(len(group[group.shot_made_flag != 1]))
        ax[x, y].title.set_text(name + score) # 성공한 슛/실패한 슛

In [ ]:
group_sf(trn_total,'shot_zone_area')

### shot 방향에 따른 성공률 차이

In [ ]:
# 왼쪽과 오른쪽의 슛 성공률의 차이가 크게 있지 않음
pd.crosstab(trn_total.shot_made_flag,trn_total.shot_zone_area).apply(lambda x: x/sum(x))

In [ ]:
group_sf(trn_total, 'shot_zone_basic')

In [ ]:
group_sf(trn_total,'shot_zone_range')

* 단변량 값으로 뚜렷하게 Success/fail이 나눠지는 값이 없음

## 2-9 Matchup, opponent

In [ ]:
print(set(trn_total.team_id))
print(set(trn_total.team_name))
print(set(trn_total.matchup)) # vs는 home // @ away
print(set(trn_total.opponent))

In [ ]:
print(trn_total.iloc[0,:].matchup)
print(trn_total.iloc[0,:].opponent)

* matchup 정보와 opponent는 동일 정보!